<h1 align=center><font size = 5>Build a Regression Model in Keras</font></h1> 

In this course project, you will build a regression model using the deep learning Keras library, and then you will experiment with increasing the number of training epochs and changing number of hidden layers and you will see how changing these parameters impacts the performance of the model.

##  Getting our Dataset Ready

We will be playing around with the same dataset that we used in the videos.

<strong>The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:</strong>

<strong>1. Cement</strong>

<strong>2. Blast Furnace Slag</strong>

<strong>3. Fly Ash</strong>

<strong>4. Water</strong>

<strong>5. Superplasticizer</strong>

<strong>6. Coarse Aggregate</strong>

<strong>7. Fine Aggregate</strong>

Let's start by importing the <em>pandas</em> and the Numpy libraries.

In [1]:
import pandas as pd
import numpy as np

Let's download the data and read it into a <em>pandas</em> dataframe.

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


So the first concrete sample has 540 cubic meter of cement, 0 cubic meter of blast furnace slag, 0 cubic meter of fly ash, 162 cubic meter of water, 2.5 cubic meter of superplaticizer, 1040 cubic meter of coarse aggregate, 676 cubic meter of fine aggregate. Such a concrete mix which is 28 days old, has a compressive strength of 79.99 MPa. 

#### Let's check how many data points we have.

In [3]:
concrete_data.shape

(1030, 9)

So, there are approximately 1000 samples to train our model on. Because of the few samples, we have to be careful not to overfit the training data.

Let's check the dataset for any missing values.

In [4]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

The data looks very clean and is ready to be used to build our model.

#### Split data into predictors and target

The target variable in this problem is the concrete sample strength. Therefore, our predictors will be all the other columns.

In [6]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Let's do a quick sanity check of the predictors and the target dataframes.

In [7]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [8]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Let's save the number of predictors to *n_cols* since we will need this number when building our network.

In [10]:
n_cols = predictors.shape[1] # number of predictors

## Import Keras

Let's go ahead and import the Keras library

In [11]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:520: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:521: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

Let's import the rest of the packages from the Keras library that we will need to build our regression model.

In [12]:
from keras.models import Sequential
from keras.layers import Dense

## Build a Neural Network

Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.

In [13]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

The above function create a model that has one hidden layers consisting of 10 hidden units. The network uses <strong>reLU</strong> as the activation function and <strong>mean squared error</strong> as the loss function

## Train and Test the Network

Let's call the function now to create our model.

In [14]:
model = regression_model()

We will import the <strong>train_test_split</strong> function from <strong>scikit-learn</strong> to randomly split the data into a training and test sets. Also, we will import the <strong>mean_squared_error</strong> function from Scikit-learn.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

Next, We will create a function called <strong>evaluate_model</strong> which will perform the following tasks.

1. Randomly split the data into a training and test sets by holding 30% of the data for testing

2. Train and test the model at the same time using the <strong>fit</strong> method. We will train the model for 50 epochs.

3. After training is complete, We will need to evaluate the model on the test data.

4. Compute the mean squared error between the predicted concrete strength and the actual concrete strength.

5. Return the mean_squared_error

In [16]:
def evaluate_model(predictors, target, test_size = 0.3, random_state = 1, epochs = 50, verbose = 1):
    # Split data 
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=test_size, random_state=random_state)
    # Fit the model
    model.fit(X_train, y_train, epochs=epochs, verbose=verbose)
    # Evaluate model
    loss_val = model.evaluate(X_test, y_test,verbose=verbose)
    y_pred = model.predict(X_test)
    #Compute the mean squared error
    mean_square_error = mean_squared_error(y_test, y_pred)
    return (loss_val, mean_square_error)

In [17]:
print(evaluate_model(predictors, target,0.3,47,50,1))

Epoch 1/50
721/721 [==============================] - 1s 847us/step - loss: 18277.9740
Epoch 2/50
721/721 [==============================] - 0s 275us/step - loss: 9413.2826
Epoch 3/50
721/721 [==============================] - 0s 282us/step - loss: 6662.1505
Epoch 4/50
721/721 [==============================] - 0s 270us/step - loss: 5152.5719
Epoch 5/50
721/721 [==============================] - 0s 334us/step - loss: 4128.6773
Epoch 6/50
721/721 [==============================] - 0s 386us/step - loss: 3347.2000
Epoch 7/50
721/721 [==============================] - 0s 361us/step - loss: 2729.0890
Epoch 8/50
721/721 [==============================] - 0s 330us/step - loss: 2292.0426
Epoch 9/50
721/721 [==============================] - 0s 311us/step - loss: 1962.3534
Epoch 10/50
721/721 [==============================] - 0s 363us/step - loss: 1694.3049
Epoch 11/50
721/721 [==============================] - 0s 412us/step - loss: 1492.1523
Epoch 12/50
721/721 [==============================

Finally, Create a list of 50 mean squared errors and report mean and the standard deviation of the mean squared errors.

In [18]:
total_mean_squared_errors = 50
epochs = 50
mean_squared_errors = []

for i in range(0, total_mean_squared_errors):
    MSE,mean_square_error = evaluate_model(predictors, target,0.3,i,epochs,0)
    print("MSE "+str(i+1)+": "+str(MSE))
    mean_squared_errors.append(mean_square_error)

mean_squared_errors = np.array(mean_squared_errors)
mean = np.mean(mean_squared_errors)
standard_deviation = np.std(mean_squared_errors)

print('\n')
print("Number of epochs for each training: "+str(epochs) )
print("Mean of "+str(total_mean_squared_errors)+" MSE: " + str(mean))
print("Standard Deviation of "+str(total_mean_squared_errors)+" MSE: " + str(standard_deviation))

MSE 1: 104.91371359717112
MSE 2: 118.29944879723213
MSE 3: 105.91366248763495
MSE 4: 89.31578468890638
MSE 5: 79.31778371372656
MSE 6: 73.08827525512301
MSE 7: 84.89949988701582
MSE 8: 72.11772308843422
MSE 9: 58.82295575188201
MSE 10: 57.150605976388675
MSE 11: 51.64214639756286
MSE 12: 52.68340015102744
MSE 13: 56.6501890941731
MSE 14: 56.8288864086361
MSE 15: 49.05159137318435
MSE 16: 43.390940243372256
MSE 17: 49.36704914623866
MSE 18: 53.95854857361432
MSE 19: 46.05285797612952
MSE 20: 50.04247934532783
MSE 21: 54.36053471734994
MSE 22: 47.280092023337154
MSE 23: 65.32532935775214
MSE 24: 52.5448498895639
MSE 25: 48.64331648496363
MSE 26: 47.5376002981439
MSE 27: 51.611056133381375
MSE 28: 44.9634034487036
MSE 29: 60.78320980380654
MSE 30: 59.729114421363015
MSE 31: 52.2843421269389
MSE 32: 43.923705721364435
MSE 33: 48.774641388828314
MSE 34: 50.36996761186223
MSE 35: 49.6940145153058
MSE 36: 53.58501894805809
MSE 37: 53.09463334315031
MSE 38: 55.32306135356619
MSE 39: 49.5097313